# Experiment 005: Strict Ensemble with 1e-15 Overlap Threshold

exp_004 failed on Kaggle with overlaps in group 002. We need stricter overlap validation.

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
from shapely.ops import unary_union
from shapely.strtree import STRtree
from decimal import Decimal, getcontext
import json
import os
from glob import glob

getcontext().prec = 30

print("Imports done")

In [ ]:
# Tree geometry
def get_tree_polygon():
    trunk_w, trunk_h = Decimal('0.15'), Decimal('0.2')
    base_w, mid_w, top_w = Decimal('0.7'), Decimal('0.4'), Decimal('0.25')
    tip_y, tier_1_y, tier_2_y, base_y = Decimal('0.8'), Decimal('0.5'), Decimal('0.25'), Decimal('0.0')
    trunk_bottom_y = -trunk_h
    vertices = [
        (float(Decimal('0.0')), float(tip_y)),
        (float(top_w / Decimal('2')), float(tier_1_y)),
        (float(top_w / Decimal('4')), float(tier_1_y)),
        (float(mid_w / Decimal('2')), float(tier_2_y)),
        (float(mid_w / Decimal('4')), float(tier_2_y)),
        (float(base_w / Decimal('2')), float(base_y)),
        (float(trunk_w / Decimal('2')), float(base_y)),
        (float(trunk_w / Decimal('2')), float(trunk_bottom_y)),
        (float(-(trunk_w / Decimal('2'))), float(trunk_bottom_y)),
        (float(-(trunk_w / Decimal('2'))), float(base_y)),
        (float(-(base_w / Decimal('2'))), float(base_y)),
        (float(-(mid_w / Decimal('4'))), float(tier_2_y)),
        (float(-(mid_w / Decimal('2'))), float(tier_2_y)),
        (float(-(top_w / Decimal('4'))), float(tier_1_y)),
        (float(-(top_w / Decimal('2'))), float(tier_1_y)),
    ]
    return Polygon(vertices)

TREE_POLY = get_tree_polygon()
print(f"Tree polygon: {len(TREE_POLY.exterior.coords)} vertices")

In [ ]:
def parse_s_value(s_val):
    if isinstance(s_val, str) and s_val.startswith('s'):
        return float(s_val[1:])
    return float(s_val)

def create_tree(x, y, deg):
    return translate(rotate(TREE_POLY, deg, origin=(0, 0)), x, y)

def get_bbox_side(polygons):
    if not polygons:
        return 0
    combined = unary_union(polygons)
    bounds = combined.bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

# STRICT overlap check - use 1e-15 threshold (stricter than before)
def check_overlaps_strict(polygons, tolerance=1e-15):
    """Check for overlaps with STRICT tolerance."""
    if len(polygons) <= 1:
        return False, None
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]):
                if not polygons[i].touches(polygons[j]):
                    intersection = polygons[i].intersection(polygons[j])
                    if intersection.area > tolerance:
                        return True, f"Trees {i} and {j} overlap (area={intersection.area:.2e})"
    return False, None

print("Functions defined with STRICT overlap check (1e-15 threshold)")

In [ ]:
def load_submission(path):
    """Load submission CSV and extract per-N data."""
    try:
        df = pd.read_csv(path)
        if 'x' not in df.columns or 'y' not in df.columns or 'deg' not in df.columns:
            return None
        df['x_val'] = df['x'].apply(parse_s_value)
        df['y_val'] = df['y'].apply(parse_s_value)
        df['deg_val'] = df['deg'].apply(parse_s_value)
        df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))
        return df
    except:
        return None

def get_n_data(df, n):
    """Get data for a specific N value."""
    n_df = df[df['n'] == n]
    if len(n_df) != n:
        return None
    return {
        'xs': n_df['x_val'].tolist(),
        'ys': n_df['y_val'].tolist(),
        'degs': n_df['deg_val'].tolist(),
        'x_strs': n_df['x'].tolist(),
        'y_strs': n_df['y'].tolist(),
        'deg_strs': n_df['deg'].tolist()
    }

def compute_n_score(xs, ys, degs):
    """Compute score for N trees."""
    n = len(xs)
    polygons = [create_tree(xs[i], ys[i], degs[i]) for i in range(n)]
    side = get_bbox_side(polygons)
    return (side ** 2) / n, polygons

print("Helper functions defined")

In [ ]:
# Load the VALID baseline (this passed Kaggle validation)
baseline_path = '/home/nonroot/snapshots/santa-2025/21328309254/submission/submission.csv'
baseline_df = load_submission(baseline_path)
print(f"Loaded baseline: {len(baseline_df)} rows")

# Compute baseline scores for each N
baseline_scores = {}
baseline_data = {}
for n in range(1, 201):
    data = get_n_data(baseline_df, n)
    if data:
        score, _ = compute_n_score(data['xs'], data['ys'], data['degs'])
        baseline_scores[n] = score
        baseline_data[n] = data

baseline_total = sum(baseline_scores.values())
print(f"Baseline total score: {baseline_total:.6f}")

In [ ]:
# Scan all CSV files in snapshots
csv_files = glob('/home/nonroot/snapshots/santa-2025/*/submission/submission.csv')
csv_files += glob('/home/nonroot/snapshots/santa-2025/*/code/*.csv')
csv_files += glob('/home/nonroot/snapshots/santa-2025/*/code/**/*.csv', recursive=True)
print(f"Found {len(csv_files)} CSV files to scan")

In [ ]:
# For each N, find the best STRICTLY VALID solution
best_per_n = {}  # n -> {'score': float, 'data': dict, 'source': str}

# Initialize with baseline (known to be valid)
for n in range(1, 201):
    best_per_n[n] = {
        'score': baseline_scores[n],
        'data': baseline_data[n],
        'source': 'baseline'
    }

print("Initialized with baseline. Now scanning for improvements...")
print("="*60)

improvement_count = 0
files_processed = 0

for csv_path in csv_files:
    df = load_submission(csv_path)
    if df is None:
        continue
    
    files_processed += 1
    if files_processed % 100 == 0:
        print(f"Processed {files_processed} files...")
    
    for n in range(1, 201):
        data = get_n_data(df, n)
        if data is None:
            continue
        
        # Compute score
        score, polygons = compute_n_score(data['xs'], data['ys'], data['degs'])
        
        # Only consider if better than current best
        if score >= best_per_n[n]['score']:
            continue
        
        # STRICT overlap check
        has_overlap, msg = check_overlaps_strict(polygons)
        if has_overlap:
            continue
        
        # Found a valid improvement!
        improvement = best_per_n[n]['score'] - score
        if improvement > 0.0001:  # Only report significant improvements
            print(f"N={n}: {best_per_n[n]['score']:.6f} -> {score:.6f} (improvement: {improvement:.6f})")
        
        best_per_n[n] = {
            'score': score,
            'data': data,
            'source': csv_path
        }
        improvement_count += 1

print(f"\nProcessed {files_processed} files")
print(f"Found {improvement_count} valid improvements")

In [ ]:
# Compute final ensemble score
ensemble_total = sum(best_per_n[n]['score'] for n in range(1, 201))
improvement = baseline_total - ensemble_total

print(f"\n{'='*60}")
print(f"Baseline score:  {baseline_total:.6f}")
print(f"Ensemble score:  {ensemble_total:.6f}")
print(f"Improvement:     {improvement:.6f}")
print(f"{'='*60}")
print(f"\nTarget: 68.888293")
print(f"Gap to target: {ensemble_total - 68.888293:.6f}")

# Count sources used
sources = {}
for n in range(1, 201):
    src = best_per_n[n]['source']
    if src not in sources:
        sources[src] = []
    sources[src].append(n)

print(f"\nSources used: {len(sources)}")
for src, ns in sorted(sources.items(), key=lambda x: -len(x[1]))[:5]:
    print(f"  {src.split('/')[-3] if 'snapshots' in src else src}: {len(ns)} N values")

In [ ]:
# FINAL VALIDATION: Check ALL N values with strict overlap check
print("\nFinal validation of entire ensemble...")
all_valid = True
for n in range(1, 201):
    data = best_per_n[n]['data']
    polygons = [create_tree(data['xs'][i], data['ys'][i], data['degs'][i]) for i in range(n)]
    has_overlap, msg = check_overlaps_strict(polygons)
    if has_overlap:
        print(f"OVERLAP at N={n}: {msg}")
        all_valid = False
        # Replace with baseline
        best_per_n[n] = {
            'score': baseline_scores[n],
            'data': baseline_data[n],
            'source': 'baseline_fallback'
        }

if all_valid:
    print("All 200 configurations pass strict overlap validation!")
else:
    # Recompute score after fallbacks
    ensemble_total = sum(best_per_n[n]['score'] for n in range(1, 201))
    print(f"\nAfter fallbacks, ensemble score: {ensemble_total:.6f}")

In [ ]:
# Save submission
print("\nSaving submission...")
rows = []
for n in range(1, 201):
    data = best_per_n[n]['data']
    for i in range(n):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': data['x_strs'][i],
            'y': data['y_strs'][i],
            'deg': data['deg_strs'][i]
        })

df_out = pd.DataFrame(rows)
df_out.to_csv('/home/code/experiments/005_strict_ensemble/submission.csv', index=False)
df_out.to_csv('/home/submission/submission.csv', index=False)
print(f"Saved {len(df_out)} rows")
print(f"First 5 rows:")
print(df_out.head())

In [ ]:
# Save metrics
metrics = {
    'cv_score': ensemble_total,
    'baseline_score': baseline_total,
    'improvement': baseline_total - ensemble_total,
    'target': 68.888293,
    'gap': ensemble_total - 68.888293,
    'sources_used': len(sources),
    'overlap_threshold': 1e-15
}

with open('/home/code/experiments/005_strict_ensemble/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"\nMetrics saved: {metrics}")